## Read CSV

In [17]:
import pandas as pd
plans = pd.read_csv('./plans.csv')
slc = pd.read_csv('./slcsp.csv')
zips = pd.read_csv('./zips.csv')
slc

,zipcode,rate
0,64148,NaN
1,67118,NaN
2,40813,NaN
3,18229,NaN
4,51012,NaN
5,79168,NaN
6,54923,NaN
7,67651,NaN
8,49448,NaN
9,27702,NaN


Remove duplicate records if any

In [5]:
zips = zips[['zipcode', 'state', 'rate_area']].sort_values(['zipcode', 'state', 'rate_area']).drop_duplicates()

Identify duplicate zips, eg: same zipcode in multiple states / rate areas.
drop these duplicate zips so the result in the final answer is N/A

In [13]:
dup_zips = zips[zips.duplicated(['zipcode'], keep=False)].sort_values(['zipcode', 'state', 'rate_area'])
uniq_zips = zips.drop(dup_zips.index.values)
uniq_zips

,zipcode,state,rate_area
30314,501,NY,8
30313,544,NY,8
51384,601,PR,1
51385,602,PR,1
51386,603,PR,1
51387,604,PR,1
51389,605,PR,1
51458,606,PR,1
51531,611,PR,1
51393,612,PR,1


Filter on Silver plans

In [7]:
grp = plans[plans['metal_level'] == 'Silver'][['state', 'rate_area', 'rate']].sort_values(['state',  'rate_area', 'rate']).drop_duplicates()
grp.head(10)

,state,rate_area,rate
1869,AK,1,580.0
2156,AK,1,597.0
14962,AK,1,604.0
5393,AK,1,635.0
20138,AK,1,638.0
8180,AK,1,664.0
9910,AK,2,568.0
16353,AK,2,569.0
16749,AK,2,585.0
4600,AK,2,592.0


Check if there are two rates for the same rate area in a state

In [8]:
grp.groupby(['state', 'rate_area']).size().reset_index(name='counts').sort_values(['counts']).head(10)

,state,rate_area,counts
382,WI,3,2
398,WV,3,4
404,WV,9,4
403,WV,8,4
402,WV,7,4
401,WV,6,4
400,WV,5,4
399,WV,4,4
405,WV,10,4
397,WV,2,4


find the index of the minimum rate plan in rate_area 
drop the lowest plans from the list

In [9]:
lowestPlanIdx = grp.groupby(['state', 'rate_area'])['rate'].idxmin().values
grp3 = grp.drop(lowestPlanIdx)

get the second lowest rateplan

In [10]:
grp4 = grp3.groupby(['state', 'rate_area'], as_index=False).min()

Join unique zipcode with the second lowest rate to get the final answer

In [12]:
res1 = pd.merge(uniq_zips, grp4, on=['state', 'rate_area'], how="left")
res2 = pd.merge(slc, res1, on=['zipcode'], how="left")
ans = res2[['zipcode', 'rate_y']]
ans

,zipcode,rate_y
0,64148,245.20
1,67118,212.35
2,40813,NaN
3,18229,231.48
4,51012,252.76
5,79168,243.68
6,54923,NaN
7,67651,249.44
8,49448,221.63
9,27702,283.08
